# Plant individual effects (radar plots)
This notebook shows how to load individual effects on plants from an xNonTargetTerrestrialPlant simulation and how to prepare "radar plots" from accordingly aggregated Assessment Endpoints using R and xRisk.

The code here makes use of the _XRisk_ R package that contains functions to access simulation data stored in X3df files that is the default data storage for xNonTargetTerrestrialPlant simulations.

In [ ]:
library(XRisk)

Many _XRisk_ functions return data tables, and it is a good idea to load the _data.table_ package to process these data tables further with high performance.

In [ ]:
library(data.table)

For plotting, _ggplot2_ is used here.

In [ ]:
library(ggplot2)

The following statement increases the size of plots within the notebook.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

The _X3DF_ class provides a simple wrapper to operate against X3df files. The sample notebook here uses an exemplary dataset in the _sample_ folder (all relative paths in the notebook are resolved from the _analysis_). To analyze custom simulation runs, e.g. from the _processing_ folder, you can pass in an absolute path instead.

In [ ]:
x3df <- X3DF$new("sample/store/arr.dat")

The `x3df` variable now contains an instance of an S4 class that wraps access to the X3df file.

In [ ]:
x3df

Of most interest here is the `x3df$datasets` binding that returns a list of datasets that are stored in the X3df file. AT this level, datasets are described by their access path, name and a selection of additional metadata like sizes and scales. To understand the individual datasets entirely, please refer to the individual component documentations.

In [ ]:
x3df$datasets

For the current assessment, we are interested in the individual plant effects as expressed by logistic dose-response curves for several plant species, entities and attributes. These effects are given as numerical values between 0 and 1 at a temporal scale of one day (though only one day, the day of application, was simulated) and for a spatial extent of 200 m x 100 m at the square-meter scale. The according datasets start with `"/DoseResponse"` as an indicator of the component that calculated them and end with `"/Effect"` indicating the specific dataset of the component. This allows to quickly filter the available datasets for the ones of interest.

In [ ]:
dose_response.datasets <- 
  x3df$datasets[startsWith(names(x3df$datasets), "DoseResponse") & endsWith(names(x3df$datasets), "/Effect")]

After applying the filter criteria, a number of datasets containing effect endpoints remain.

In [ ]:
length(dose_response.datasets)

With all this preparation, we can start to read the actual data from the X3df file. There are several ways to do so, but in the current example we can make use of the fact that the schematic scenario consists of a 100 m x 100 m field and an adjacent 100 m x 100 m habitat to the East. If we are interested in all one square-meter cells that are located 3 m away from the field edge, we have to look for all cells with an x-coordinate of 103 m. We make use here of the dataset's `extract` method to retrieve all value of the first (and only) simulated day that are located in cells with an x-coordinate of 103 m and having any y-coordinate. This is done, as an example, for the first effect dataset.

In other simulations, additional data would be necessary to indicate which cells are located 3 m away from the field edge.

In [ ]:
ds <- dose_response.datasets[[1]]
names(dose_response.datasets)[1]
ds$extract(c(1, 103, NA))

The extraction procedure can be easily applied to all effect datasets to yield a list of data tables containing all effects that were simulated in the sample simulation 3 m away of the field edge.

In [ ]:
data <- lapply(dose_response.datasets, function(ds) ds$extract(c(1, 103, NA)))

The list is rather long but nonetheless already filtered enough to be handled entirely. For large simulations, however, values should be preferably further aggregated during reading the data (using `chunk-apply` instead of `extract`) to reduce the amount of data points stored in memory.

In [ ]:
data

This dataset can already be used for many assessments. In this notebook, we are, for instance, further interested in the maximum, median and 90th percentile of effects observed 3 m away from field edge. For this reason, we now calculate the according effects for each individual plant endpoint.

In [ ]:
effects <- rbindlist(lapply(data, function(d) d[, as.list(quantile(value, c(0.5, 0.9, 1)))]), idcol = ,"endpoint")

The `rbindlist` in the previous command combined the results for the individual endpoints into a single data table. Without `rbindlist`, the result would have been a list of values. The `idcol` argument makes sure that the name of the individual endpoint is also stored in the resulting data table.

In [ ]:
effects

Before proceeding, we clean up the endpoint names by removing the common prefixes and suffixes and by replacing underscores with spaces.

In [ ]:
effects[, endpoint := gsub("_", " ", substr(endpoint, 14, nchar(endpoint) - 7), TRUE)]

The table is now much more readable:

In [ ]:
effects

For plotting (and potential further aggregation), it is useful to separate species and attribute from the name of the endpoint. This is done with the next few lines of code.

In [ ]:
effects <- effects[
    , 
    .(species = paste(strsplit(endpoint, " ", TRUE)[[1]][1], strsplit(endpoint, " ", TRUE)[[1]][2])), 
    .(endpoint, `50%`, `90%`, `100%`)
]
effects[, attribute := substring(endpoint, nchar(species) + 1, nchar(endpoint))]
effects

Finally, a radar plot showing the individual maximum effects can be created. A red dot marks the maximum effect observed over all endpoints.

In [ ]:
ggplot(effects, aes(endpoint)) +
  geom_bar(aes(fill = species), size = 0, alpha = .5) +  
  geom_hline(yintercept = c(0.25, 0.5, 0.75), color = "white", size = 1) +
  geom_hline(yintercept = 1) +
  geom_text(data = data.table(value = c(0.25, 0.5)), aes(label = value, y = value), x = 1, size = 5, alpha = .75) +
  geom_text(
      aes(label = attribute, angle = seq(-90, -450,  length.out = effects[, .N])), 
      size = 5, 
      y = 0.8,
      alpha = 0.5
  ) +
  geom_linerange(aes(ymax = `100%`), ymin = 0, size = 1.5) +
  geom_point(data = effects[`100%` == max(`100%`)], aes(y = `100%`), color = "darkred", size = 5) + 
  coord_polar() +
  scale_y_continuous(breaks = c(limits = c(0, 1), 0.25, 0.5, 0.75)) +  
  scale_fill_brewer(palette = "Set3") +
  ggtitle(
      "Maximum effects for individual plant endpoints in 3 m distance from the field edge for a single application "
  ) +
  theme_bw() +
  theme(
      axis.text = element_blank(),
      axis.title = element_blank(),      
      panel.grid = element_blank(),      
      legend.title = element_text(size = 24),
      legend.text = element_text(size = 18),
      plot.title = element_text(size = 20, face = "bold")
  )

The same code can be used to plot the 90% percentile of individual effects

In [ ]:
ggplot(effects, aes(endpoint)) +
  geom_bar(aes(fill = species), size = 0, alpha = .5) +  
  geom_hline(yintercept = c(0.25, 0.5, 0.75), color = "white", size = 1) +
  geom_hline(yintercept = 1) +
  geom_text(data = data.table(value = c(0.25, 0.5)), aes(label = value, y = value), x = 1, size = 5, alpha = .75) +
  geom_text(
      aes(label = attribute, angle = seq(-90, -450,  length.out = effects[, .N])), 
      size = 5, 
      y = 0.8,
      alpha = 0.5
  ) +
  geom_linerange(aes(ymax = `90%`), ymin = 0, size = 1.5) +
  geom_point(data = effects[`100%` == max(`100%`)], aes(y = `100%`), color = "darkred", size = 5) + 
  coord_polar() +
  scale_y_continuous(breaks = c(limits = c(0, 1), 0.25, 0.5, 0.75)) +  
  scale_fill_brewer(palette = "Set3") +
  ggtitle(
      "90% of effects for individual plant endpoints in 3 m distance from the field edge for a single application "
  ) +
  theme_bw() +
  theme(
      axis.text = element_blank(),
      axis.title = element_blank(),      
      panel.grid = element_blank(),      
      legend.title = element_text(size = 24),
      legend.text = element_text(size = 18),
      plot.title = element_text(size = 20, face = "bold")
  )

And finally, the median effect.

In [ ]:
ggplot(effects, aes(endpoint)) +
  geom_bar(aes(fill = species), size = 0, alpha = .5) +  
  geom_hline(yintercept = c(0.25, 0.5, 0.75), color = "white", size = 1) +
  geom_hline(yintercept = 1) +
  geom_text(data = data.table(value = c(0.25, 0.5)), aes(label = value, y = value), x = 1, size = 5, alpha = .75) +
  geom_text(
      aes(label = attribute, angle = seq(-90, -450,  length.out = effects[, .N])), 
      size = 5, 
      y = 0.8,
      alpha = 0.5
  ) +
  geom_linerange(aes(ymax = `50%`), ymin = 0, size = 1.5) +
  geom_point(data = effects[`100%` == max(`100%`)], aes(y = `100%`), color = "darkred", size = 5) + 
  coord_polar() +
  scale_y_continuous(breaks = c(limits = c(0, 1), 0.25, 0.5, 0.75)) +  
  scale_fill_brewer(palette = "Set3") +
  ggtitle(
      "Median effects for individual plant endpoints in 3 m distance from the field edge for a single application "
  ) +
  theme_bw() +
  theme(
      axis.text = element_blank(),
      axis.title = element_blank(),      
      panel.grid = element_blank(),      
      legend.title = element_text(size = 24),
      legend.text = element_text(size = 18),
      plot.title = element_text(size = 20, face = "bold")
  )